<a href="https://colab.research.google.com/github/bhstoller/multi-agent-customer-service/blob/main/agent_to_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/bhstoller/multi-agent-customer-service.git

Cloning into 'multi-agent-customer-service'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 82 (delta 34), reused 54 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (82/82), 45.73 KiB | 6.53 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [2]:
!pip install uv -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 45.6 MB/s eta 0:00:00


In [3]:
# Install required packages
%pip install --upgrade -q google-genai google-adk==1.9.0 a2a-sdk==0.3.0 python-dotenv aiohttp uvicorn requests mermaid-python nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [4]:
import sys

from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver


class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver


patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module  # type: ignore

In [5]:
import sys

from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver
import asyncio
import logging
import os
import sys
import threading
import time

from typing import Any

import httpx
import nest_asyncio
import uvicorn

from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from dotenv import load_dotenv
from google.adk.a2a.executor.a2a_agent_executor import (
    A2aAgentExecutor,
    A2aAgentExecutorConfig,
)
from google.adk.agents import Agent, SequentialAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset
from google.adk.tools.mcp_tool.mcp_toolset import StreamableHTTPConnectionParams


/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


In [6]:
# Set Google Cloud Configuration
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'app-ai-a5'
os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'

load_dotenv()
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("a5-key")

print('Environment variables configured:')
print(f'GOOGLE_GENAI_USE_VERTEXAI: {os.environ["GOOGLE_GENAI_USE_VERTEXAI"]}')
print(f'GOOGLE_CLOUD_PROJECT: {os.environ["GOOGLE_CLOUD_PROJECT"]}')
print(f'GOOGLE_CLOUD_LOCATION: {os.environ["GOOGLE_CLOUD_LOCATION"]}')
print("GOOGLE_API_KEY loaded =", os.environ["GOOGLE_API_KEY"] is not None)

Environment variables configured:
GOOGLE_GENAI_USE_VERTEXAI: FALSE
GOOGLE_CLOUD_PROJECT: app-ai-a5
GOOGLE_CLOUD_LOCATION: us-central1
GOOGLE_API_KEY loaded = True


In [7]:
if 'google.colab' in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=os.environ['GOOGLE_CLOUD_PROJECT'])

In [8]:
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    force=True   # <-- optional, but recommended to override ADK logging
)

def log(msg: str):
    logging.getLogger("router").error(msg)


def on_before(agent, ctx):
    try:
        user_msg = ctx.input_messages[0].content
    except:
        user_msg = None

    log(f"Router → {agent.name}: sending query: {user_msg}")

def on_after(agent, ctx, result_text):
    log(f"{agent.name} → Router: {result_text}")


In [9]:
class LoggingRouter(SequentialAgent):
    async def _run_async_impl(self, ctx):

        # -------------- FIX: Extract user text safely ------------------
        try:
            # ADK always provides input_messages
            user_msg = ctx.input_messages[0].content
        except Exception:
            user_msg = None

        log(f"Router received user query: {user_msg}")

        final_output = ""

        # Iterate through sub-agents
        for agent in self.sub_agents:

            log(f"Router → {agent.name}: forwarding query")

            collected = ""

            # Stream all events from sub-agent
            async for event in agent.run_async(ctx):

                # Capture deltas for logging
                if hasattr(event, "delta") and event.delta:
                    collected += event.delta

                # Must forward event
                yield event

            log(f"{agent.name} → Router: {collected}")
            final_output = collected

        # -------------- FIX: Use AgentTextEvent for final response -----
        yield AgentTextEvent(text=final_output)

# **Add Agents**

In [10]:
MCP_SERVER_URL = userdata.get('MCP_SERVER_URL')
print(MCP_SERVER_URL)

https://polar-nonsolubly-madden.ngrok-free.dev/mcp


## Add Customer Agent

In [11]:
customer_data_agent = Agent(
    model="gemini-2.0-flash",
    name="customer_data_agent",
    tools=[ MCPToolset(
        connection_params=StreamableHTTPConnectionParams(
            url=MCP_SERVER_URL
            )
        )
    ],
    instruction="""
    You are the Customer Data Agent.

    Your role is to interface with the MCP server and perform EXACT data operations.

    REQUIRED RULES:
    - ALWAYS call an MCP tool for any operation involving customer data.
    - NEVER invent fields, values, or IDs.
    - ALWAYS return valid JSON.
    - If a customer does not exist, return:
      {"error": "Customer ID not found", "customer_id": <id>}
    - For successful lookups, return:
      {"customer": { ...mcp result... }}

    AVAILABLE OPERATIONS:
    - get_customer(customer_id)
    - list_customers(status, limit)
    - update_customer(customer_id, data)
    - create_ticket(customer_id, issue, priority)
    - get_customer_history(customer_id)

    ADDITIONAL RULES:
    - When updating a customer, preserve fields not being modified.
    - When creating a ticket, always include created_at returned by MCP.
    - When listing customers, return array of customer objects.

    DO NOT return natural language. ONLY JSON.
    """
)
print("Customer Data Agent created!")

Customer Data Agent created!


In [12]:
customer_data_agent_card = AgentCard(
    name='Customer Data Agent',
    url='http://127.0.0.1:10020',
    description='Fetches and updates customer data using MCP tools',
    version='1.0',
    capabilities=AgentCapabilities(streaming=False),
    default_input_modes=['text/plain'],
    default_output_modes=['application/json'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='customer_data_access',
            name='Customer Data Access',
            description='Retrieve and update customer records',
            tags=['customer', 'data', 'database', 'lookup'],
            examples=[
                'Get customer 42',
                'Update customer 5 email',
                'Show all customers',
            ],
        )
    ],
)


In [13]:
remote_customer_data_agent = RemoteA2aAgent(
    name='customer_data_agent',
    description='Remote wrapper for Customer Data Agent',
    agent_card=f'http://127.0.0.1:10020{AGENT_CARD_WELL_KNOWN_PATH}',  # ← FIXED
)

/tmp/ipython-input-2030816957.py:1: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  remote_customer_data_agent = RemoteA2aAgent(


## Specialist Agent

In [14]:
support_agent = Agent(
    model="gemini-2.0-flash-exp",
    name="support_agent",
    instruction="""
    You are the Support Agent.
    RULES:
    - DO NOT infer or create customer attributes (address, tier, status, balance, etc.)
    - DO NOT guess missing information.
    - ONLY use customer data exactly as provided by the Router.
    - If customer-specific data is required to answer, respond ONLY with:

    {
      "needs_customer_data": true,
      "reason": "<why>",
      "requested_fields": ["email", "tickets", "status", ...]
    }

    When the Router provides customer data:
    - Use only those fields.
    - Provide a natural-language support answer.
    - Do not return JSON unless explicitly requested by the Router.

    Your job: troubleshooting, escalation, and answering support questions.
    """,
)

In [15]:
support_agent_card = AgentCard(
    name='Support Agent',
    url='http://127.0.0.1:10021',   # FIXED
    description='Handles general support questions and escalates to customer data agent when needed',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id='customer_support',
            name='Customer Support',
            description='Provides general support responses',
            tags=['support', 'help', 'troubleshooting'],
            examples=[
                'How do I reset my password?',
                'What is your refund policy?',
                'I need help with my account.',
            ],
        )
    ],
)


In [16]:
remote_support_agent = RemoteA2aAgent(
    name='support_agent',
    description='Remote wrapper for Support Agent',
    agent_card=f'http://127.0.0.1:10021{AGENT_CARD_WELL_KNOWN_PATH}',  # FIXED
)

/tmp/ipython-input-332038127.py:1: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  remote_support_agent = RemoteA2aAgent(


## Router Agent

In [18]:
router_agent = SequentialAgent(
    name="router_agent_host",
    sub_agents=[customer_data_agent, support_agent]
)

In [19]:
router_agent_card = AgentCard(
    name="router_agent_host",
    url="http://localhost:10040",
    description="orchestrates customer data agent and support agent, routes to support and customer-data agents, and composes the final answer.",
    version="1.0",
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=["text/plain"],
    default_output_modes=["application/json"],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id="router_orchestration",
            name="Router / Orchestrator",
            description="Routes user queries to sub-agents using A2A calls and coordinates outputs.",
            tags=["router", "orchestrator", "workflow", "a2a"],
            examples=[
                "Help me access my account",
                "Look up customer 12",
                "Update the customer's email",
                "The customer cannot log in",
            ]
        )
    ]
)

# **Running**

In [20]:
def create_agent_a2a_server(agent, agent_card):
    """Create an A2A server for any ADK agent.

    Args:
        agent: The ADK agent instance
        agent_card: The ADK agent card

    Returns:
        A2AStarletteApplication instance
    """
    runner = Runner(
        app_name=agent.name,
        agent=agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),
    )

    config = A2aAgentExecutorConfig()
    executor = A2aAgentExecutor(runner=runner, config=config)

    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )

    # Create A2A application
    return A2AStarletteApplication(
        agent_card=agent_card, http_handler=request_handler
    )

In [21]:
# Apply nest_asyncio
nest_asyncio.apply()

# Store server tasks
server_tasks: list[asyncio.Task] = []


async def run_agent_server(agent, agent_card, port) -> None:
    """Run a single agent server."""
    app = create_agent_a2a_server(agent, agent_card)

    config = uvicorn.Config(
        app.build(),
        host='127.0.0.1',
        port=port,
        log_level='warning',
        loop='none',  # Important: let uvicorn use the current loop
    )

    server = uvicorn.Server(config)
    await server.serve()


async def start_all_servers() -> None:
    """Start all servers in the same event loop."""
    tasks = [
        asyncio.create_task(
            run_agent_server(customer_data_agent, customer_data_agent_card, 10020)
        ),
        asyncio.create_task(
            run_agent_server(support_agent, support_agent_card, 10021)
        ),
        asyncio.create_task(
            run_agent_server(router_agent, router_agent_card, 10040)
        ),
    ]

    await asyncio.sleep(2)

    print('All agent servers started!')
    print('   - Customer Data Agent: http://0.0.0.0:10020')
    print('   - Support Agent:       http://0.0.0.0:10021')
    print('   - Router Agent:        http://0.0.0.0:10040')

    try:
        # await asyncio.gather(*tasks)
        await asyncio.Event().wait()
    except KeyboardInterrupt:
        print('Shutting down servers...')


def run_servers_in_background() -> None:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_all_servers())


server_thread = threading.Thread(target=run_servers_in_background, daemon=True)
server_thread.start()

time.sleep(3)


/tmp/ipython-input-2220589873.py:19: UserWarning: [EXPERIMENTAL] A2aAgentExecutorConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  config = A2aAgentExecutorConfig()
/tmp/ipython-input-2220589873.py:20: UserWarning: [EXPERIMENTAL] A2aAgentExecutor: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  executor = A2aAgentExecutor(runner=runner, config=config)


All agent servers started!
   - Customer Data Agent: http://0.0.0.0:10020
   - Support Agent:       http://0.0.0.0:10021
   - Router Agent:        http://0.0.0.0:10040


In [22]:
print('Customer Data Agent Card:')
print(customer_data_agent_card)
print('Support Agent Card:')
print(support_agent_card)
print('Host Agent Card:')
print(router_agent_card)

Customer Data Agent Card:
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=False) default_input_modes=['text/plain'] default_output_modes=['application/json'] description='Fetches and updates customer data using MCP tools' documentation_url=None icon_url=None name='Customer Data Agent' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Retrieve and update customer records', examples=['Get customer 42', 'Update customer 5 email', 'Show all customers'], id='customer_data_access', input_modes=None, name='Customer Data Access', output_modes=None, security=None, tags=['customer', 'data', 'database', 'lookup'])] supports_authenticated_extended_card=None url='http://127.0.0.1:10020' version='1.0'
Support Agent Card:
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notific

In [23]:
class A2ASimpleClient:
    """A2A Simple to call A2A servers."""

    def __init__(self, default_timeout: float = 240.0):
        self._agent_info_cache: dict[
            str, dict[str, Any] | None
        ] = {}  # Cache for agent metadata
        self.default_timeout = default_timeout

    async def create_task(self, agent_url: str, message: str) -> str:
        """Send a message following the official A2A SDK pattern."""
        # Configure httpx client with timeout
        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0,
            read=self.default_timeout,
            write=10.0,
            pool=5.0,
        )

        async with httpx.AsyncClient(timeout=timeout_config) as httpx_client:
            # Check if we have cached agent card data
            if (
                agent_url in self._agent_info_cache
                and self._agent_info_cache[agent_url] is not None
            ):
                agent_card_data = self._agent_info_cache[agent_url]
            else:
                # Fetch the agent card
                agent_card_response = await httpx_client.get(
                    f'{agent_url}{AGENT_CARD_WELL_KNOWN_PATH}'
                )
                agent_card_data = self._agent_info_cache[agent_url] = (
                    agent_card_response.json()
                )

            # Create AgentCard from data
            agent_card = AgentCard(**agent_card_data)

            # Create A2A client with the agent card
            config = ClientConfig(
                httpx_client=httpx_client,
                supported_transports=[
                    TransportProtocol.jsonrpc,
                    TransportProtocol.http_json,
                ],
                use_client_preference=True,
            )

            factory = ClientFactory(config)
            client = factory.create(agent_card)

            # Create the message object
            message_obj = create_text_message_object(content=message)

            # Send the message and collect responses
            responses = []
            async for response in client.send_message(message_obj):
                responses.append(response)

            # The response is a tuple - get the first element (Task object)
            if (
                responses
                and isinstance(responses[0], tuple)
                and len(responses[0]) > 0
            ):
                task = responses[0][0]  # First element of the tuple

                # Extract text: task.artifacts[0].parts[0].root.text
                try:
                    return task.artifacts[0].parts[0].root.text
                except (AttributeError, IndexError):
                    return str(task)

            return 'No response received'

In [24]:
a2a_client = A2ASimpleClient()

In [25]:
async def test_customer_data():
    response = await a2a_client.create_task(
        'http://127.0.0.1:10020',  # correct
        'Look up customer with id 1'
    )
    print(response)

asyncio.run(test_customer_data())

/usr/local/lib/python3.12/dist-packages/google/adk/a2a/executor/a2a_agent_executor.py:184: UserWarning: [EXPERIMENTAL] convert_a2a_request_to_adk_run_args: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  run_args = convert_a2a_request_to_adk_run_args(context)
/usr/local/lib/python3.12/dist-packages/google/adk/a2a/converters/request_converter.py:64: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  convert_a2a_part_to_genai_part(part)
/usr/local/lib/python3.12/dist-packages/google/adk/a2a/executor/a2a_agent_executor.py:214: UserWarning: [EXPERIMENTAL] TaskResultAggregator: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  task_result_aggregator = TaskResu

```json
{"customer": {"id": 1, "name": "John Doe", "email": "new-email@example.com", "phone": "+1-555-0101", "status": "active", "created_at": "2025-11-17 21:35:02", "updated_at": "2025-11-18 04:02:52"}}
```


In [26]:
async def test_support_agent():
    response = await a2a_client.create_task(
        'http://localhost:10021',  # correct
        'A customer says they cannot log in. What should they do?'
    )
    print("Support Agent Response:\n", response)

asyncio.run(test_support_agent())

Support Agent Response:
 Please try resetting your password using the "Forgot Password" link on the login page. If you continue to experience difficulties, please contact support for further assistance.



In [27]:
async def test_router_agent():
    response = await a2a_client.create_task(
        'http://localhost:10040',
        'Update customer 1 email to new-email@example.com'
    )
    print("Router Agent Response:\n", response)

asyncio.run(test_router_agent())

2025-11-18 04:08:14,953 - ERROR - asyncio - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7b6ce1d18470>, 455.169682269)])']
connector: <aiohttp.connector.TCPConnector object at 0x7b6ce202b440>


Router Agent Response:
 OK, I've updated the email for John Doe (customer ID 1) to new-email@example.com.



In [28]:
async def test_full_system():
    response = await a2a_client.create_task(
        'http://localhost:10040',   # ROUTER ONLY
        'I need help with my account, customer ID 1'
    )
    print("Full System Response:\n", response)

asyncio.run(test_full_system())

2025-11-18 04:08:21,059 - ERROR - asyncio - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7b6ce1d18950>, 456.389379648)])']
connector: <aiohttp.connector.TCPConnector object at 0x7b6ce21177d0>
2025-11-18 04:08:25,289 - ERROR - asyncio - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7b6ce1d19430>, 465.506362914)])']
connector: <aiohttp.connector.TCPConnector object at 0x7b6ce202e000>


Full System Response:
 OK. I see that a high-priority ticket (ID 27) has been created for you regarding help with your account. The ticket is currently open. How can I further assist you with your account issue?



In [30]:
async def test_full_system():
    response = await a2a_client.create_task(
        'http://localhost:10040',
        "I want to cancel my subscription but I'm having billing issues"
    )
    print("Full System Response:\n", response)

asyncio.run(test_full_system())

2025-11-18 04:10:11,149 - ERROR - asyncio - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7b6ce2205a30>
2025-11-18 04:10:11,151 - ERROR - asyncio - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7b6ce1d1a030>, 570.782616192)])']
connector: <aiohttp.connector.TCPConnector object at 0x7b6ce2116510>


Full System Response:
 ```json
{
  "needs_customer_data": true,
  "reason": "I need the customer ID to create a ticket and determine the priority based on the customer's subscription and billing issues.",
  "requested_fields": ["customer_id"]
}
```



In [33]:
async def test_full_system():
    response = await a2a_client.create_task(
        'http://localhost:10040',
        "What's the status of all high-priority tickets?"
    )
    print("Full System Response:\n", response)

asyncio.run(test_full_system())

2025-11-18 04:11:38,874 - ERROR - asyncio - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x7b6ce1d1b650>, 650.319560205)])']
connector: <aiohttp.connector.TCPConnector object at 0x7b6ce2113830>


Full System Response:
 ```json
{
  "needs_customer_data": true,
  "reason": "I need access to ticket information to check the status of high-priority tickets.",
  "requested_fields": []
}
```
